preProcessing

Description: Creating the Word2Vec models for fake and real reviews from 600K yelp database (classified) from Kaggle

1. Remove stop-words, numbers, non-alphabet,...
2. Create w2v models

Data source:
@inproceedings{DBLP:conf/sigkdd/Akoglu15,
author = {Shebuti Rayana and Leman Akoglu},
title = {Collective Opinion Spam Detection: Bridging Review Networks and metadata},
booktitle = {Proceeding of the 21st ACM SIGKDD international conference
on Knowledge discovery and data mining, {KDD’15}},
year = {2015},
}


In [1]:
import pandas as pd
import json
import os
import pickle
import numpy as np

In [5]:
import re  # For preprocessing
import pandas as pd  # For data handling
from time import time  # To time our operations
from collections import defaultdict  # For word frequency

import spacy  # For preprocessing

import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [6]:
nlp = spacy.load('en_core_web_md', disable = ['parser','ner'])
def cleaning(doc):
    # Lemmatizes and removes stopwords
    # doc needs to be a spacy Doc object
    txt = [token.lemma_ for token in doc if not token.is_stop]
    # Word2Vec uses context words to learn the vector representation of a target word,
    # if a sentence is only one or two words long,
    # the benefit for the training is very small
    if len(txt) > 2:
        return ' '.join(txt)

In [13]:
from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec
from gensim.test.utils import get_tmpfile
from gensim.models import KeyedVectors
import multiprocessing

INFO - 18:05:19: adding document #0 to Dictionary(0 unique tokens: [])
INFO - 18:05:19: built Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...) from 9 documents (total 29 corpus positions)
INFO - 18:05:19: Dictionary lifecycle event {'msg': "built Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...) from 9 documents (total 29 corpus positions)", 'datetime': '2022-01-26T18:05:19.990418', 'gensim': '4.1.2', 'python': '3.8.6 (tags/v3.8.6:db45529, Sep 23 2020, 15:52:53) [MSC v.1927 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'created'}


In [ ]:
#use encode in open files to explicitly choose the codes
def splitBigJson(jsonFile, sizeSplit=100000):
#     try:
    with open(jsonFile, encoding="utf8") as file:
        data = [json.loads(line) for line in file]
    fileOriginal = jsonFile
#     sizeSplit=100000
    print('JSON size:', len(data))
    sizeSplit = int(input('Input chunk size (100000):'))
    if sizeSplit == 0:
        return False
    total = int(len(data) /sizeSplit) + 1
    for i in range(total):
        filePath = os.path.splitext(fileOriginal)[0] + str(i) + '.json'
        print(filePath)
    #     json.dump(data[i * sizeSplit:(i + 1) * sizeSplit], open(filePath, 'w',encoding='utf8'), ensure_ascii=False, indent=True)
        json.dump(data[i * sizeSplit:(i + 1) * sizeSplit], open(filePath, 'w',encoding='utf8'))
    return True
#     except:
#         print('failed')
#         return False

START

Create subset data
    each chunk has 500K review

Read the 135K review

In [2]:
dfYelp = pd.read_pickle('yelp-600K-review.pkl', compression='gzip')

dfYelp

In [ ]:
dfFake = dfYelp[dfYelp['realFake']==-1].copy()

In [3]:
dfReal = dfYelp[dfYelp['realFake']==1].copy()

In [4]:
dfReview = dfReal

Remove unwanted words
- Stop words and lemma funct
- Remove number and non-alphabet

In [7]:
brief_cleaning = (re.sub("[^A-Za-z']+", ' ', str(row)).lower().replace('\d+', '') for row in dfReview['text'])

In [9]:
txt = []
count = 0
start = time()
with nlp.select_pipes(enable="lemmatizer"):
    for doc in nlp.pipe(brief_cleaning, batch_size=1000, n_process=-1):
        if count % 50000 == 0:
            print('processed:', count, 'Time {} mins'.format(round((time() - start) / 60, 2)))
        count += 1
        txt.append(cleaning(doc))

processed: 0 Time 1.01 mins
processed: 50000 Time 2.23 mins
processed: 100000 Time 3.41 mins
processed: 150000 Time 4.61 mins
processed: 200000 Time 5.78 mins
processed: 250000 Time 6.94 mins
processed: 300000 Time 8.14 mins
processed: 350000 Time 9.31 mins
processed: 400000 Time 10.51 mins
processed: 450000 Time 11.68 mins
processed: 500000 Time 12.86 mins


In [10]:
df_clean = pd.DataFrame({'clean': txt})

In [12]:
df_clean.to_pickle('dfReal-clean.pkl', compression='gzip')

In [14]:
# Phrase & bigram
sent = [str(row).split() for row in df_clean['clean']]
phrases = Phrases(sent, min_count=1, progress_per=10000)
bigram = Phraser(phrases)
sentences = bigram[sent]

INFO - 18:05:32: collecting all words and their counts
INFO - 18:05:32: PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO - 18:05:33: PROGRESS: at sentence #10000, processed 516788 words and 343209 word types
INFO - 18:05:34: PROGRESS: at sentence #20000, processed 1071560 words and 608004 word types
INFO - 18:05:34: PROGRESS: at sentence #30000, processed 1636945 words and 860548 word types
INFO - 18:05:35: PROGRESS: at sentence #40000, processed 2159248 words and 1075278 word types
INFO - 18:05:36: PROGRESS: at sentence #50000, processed 2689731 words and 1278065 word types
INFO - 18:05:37: PROGRESS: at sentence #60000, processed 3208581 words and 1465388 word types
INFO - 18:05:38: PROGRESS: at sentence #70000, processed 3742411 words and 1647147 word types
INFO - 18:05:38: PROGRESS: at sentence #80000, processed 4276375 words and 1831994 word types
INFO - 18:05:39: PROGRESS: at sentence #90000, processed 4800011 words and 2006428 word types
INFO - 18:05:40: PROGRESS

In [15]:
#Build Word2Vec vocabularies
w2v_model = Word2Vec(min_count=3,
                     window=4,
                     vector_size=100,
                     sample=1e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=5,
                     sg=1,
                     workers=multiprocessing.cpu_count()-1)

start = time()

w2v_model.build_vocab(sentences, progress_per=50000)

# print('Time to build vocab: {} mins'.format(round((time() - start) / 60, 2)))

INFO - 18:06:39: Word2Vec lifecycle event {'params': 'Word2Vec(vocab=0, vector_size=100, alpha=0.03)', 'datetime': '2022-01-26T18:06:39.505242', 'gensim': '4.1.2', 'python': '3.8.6 (tags/v3.8.6:db45529, Sep 23 2020, 15:52:53) [MSC v.1927 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'created'}
INFO - 18:06:39: collecting all words and their counts
INFO - 18:06:39: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 18:06:43: PROGRESS: at sentence #50000, processed 2384819 words, keeping 101650 word types
INFO - 18:06:46: PROGRESS: at sentence #100000, processed 4744268 words, keeping 149834 word types
INFO - 18:06:49: PROGRESS: at sentence #150000, processed 7088063 words, keeping 183239 word types
INFO - 18:06:52: PROGRESS: at sentence #200000, processed 9422790 words, keeping 212010 word types
INFO - 18:06:55: PROGRESS: at sentence #250000, processed 11710259 words, keeping 235602 word types
INFO - 18:06:59: PROGRESS: at sentence #300000, pr

In [16]:
start = time()

w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=60, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - start) / 60, 2)))

w2v_model.init_sims(replace=True)

INFO - 18:15:05: Word2Vec lifecycle event {'msg': 'training model with 7 workers on 153773 vocabulary and 100 features, using sg=0 hs=0 sample=1e-05 negative=20 window=4 shrink_windows=True', 'datetime': '2022-01-26T18:15:05.542292', 'gensim': '4.1.2', 'python': '3.8.6 (tags/v3.8.6:db45529, Sep 23 2020, 15:52:53) [MSC v.1927 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'train'}
INFO - 18:15:06: EPOCH 1 - PROGRESS: at 1.79% examples, 137903 words/s, in_qsize 13, out_qsize 4
INFO - 18:15:07: EPOCH 1 - PROGRESS: at 4.05% examples, 167214 words/s, in_qsize 14, out_qsize 0
INFO - 18:15:08: EPOCH 1 - PROGRESS: at 6.36% examples, 177381 words/s, in_qsize 14, out_qsize 1
INFO - 18:15:09: EPOCH 1 - PROGRESS: at 8.64% examples, 178567 words/s, in_qsize 12, out_qsize 6
INFO - 18:15:10: EPOCH 1 - PROGRESS: at 11.10% examples, 183700 words/s, in_qsize 11, out_qsize 2
INFO - 18:15:11: EPOCH 1 - PROGRESS: at 13.42% examples, 183721 words/s, in_qsize 12, out_qsize 6
INFO - 18:15

INFO - 18:16:22: EPOCH 2 - PROGRESS: at 72.28% examples, 184328 words/s, in_qsize 14, out_qsize 0
INFO - 18:16:23: EPOCH 2 - PROGRESS: at 74.58% examples, 184458 words/s, in_qsize 13, out_qsize 0
INFO - 18:16:24: EPOCH 2 - PROGRESS: at 77.02% examples, 184421 words/s, in_qsize 14, out_qsize 6
INFO - 18:16:25: EPOCH 2 - PROGRESS: at 79.54% examples, 184907 words/s, in_qsize 10, out_qsize 8
INFO - 18:16:26: EPOCH 2 - PROGRESS: at 82.05% examples, 185414 words/s, in_qsize 14, out_qsize 0
INFO - 18:16:27: EPOCH 2 - PROGRESS: at 84.44% examples, 185684 words/s, in_qsize 11, out_qsize 3
INFO - 18:16:28: EPOCH 2 - PROGRESS: at 86.75% examples, 185969 words/s, in_qsize 14, out_qsize 2
INFO - 18:16:29: EPOCH 2 - PROGRESS: at 88.93% examples, 186506 words/s, in_qsize 13, out_qsize 0
INFO - 18:16:30: EPOCH 2 - PROGRESS: at 91.33% examples, 187044 words/s, in_qsize 13, out_qsize 0
INFO - 18:16:31: EPOCH 2 - PROGRESS: at 93.72% examples, 186705 words/s, in_qsize 13, out_qsize 6
INFO - 18:16:32: EPO

INFO - 18:17:37: EPOCH 4 - PROGRESS: at 45.24% examples, 195393 words/s, in_qsize 12, out_qsize 4
INFO - 18:17:38: EPOCH 4 - PROGRESS: at 47.79% examples, 195971 words/s, in_qsize 14, out_qsize 0
INFO - 18:17:39: EPOCH 4 - PROGRESS: at 50.12% examples, 195952 words/s, in_qsize 14, out_qsize 0
INFO - 18:17:40: EPOCH 4 - PROGRESS: at 52.44% examples, 195248 words/s, in_qsize 13, out_qsize 6
INFO - 18:17:41: EPOCH 4 - PROGRESS: at 54.94% examples, 196253 words/s, in_qsize 14, out_qsize 0
INFO - 18:17:42: EPOCH 4 - PROGRESS: at 57.43% examples, 196192 words/s, in_qsize 13, out_qsize 0
INFO - 18:17:43: EPOCH 4 - PROGRESS: at 59.87% examples, 196133 words/s, in_qsize 13, out_qsize 0
INFO - 18:17:44: EPOCH 4 - PROGRESS: at 62.31% examples, 195561 words/s, in_qsize 12, out_qsize 6
INFO - 18:17:45: EPOCH 4 - PROGRESS: at 64.94% examples, 196391 words/s, in_qsize 8, out_qsize 0
INFO - 18:17:46: EPOCH 4 - PROGRESS: at 67.07% examples, 195473 words/s, in_qsize 11, out_qsize 3
INFO - 18:17:47: EPOC

INFO - 18:18:52: EPOCH 6 - PROGRESS: at 18.49% examples, 194521 words/s, in_qsize 13, out_qsize 1
INFO - 18:18:53: EPOCH 6 - PROGRESS: at 20.79% examples, 192922 words/s, in_qsize 14, out_qsize 5
INFO - 18:18:54: EPOCH 6 - PROGRESS: at 23.40% examples, 194148 words/s, in_qsize 10, out_qsize 4
INFO - 18:18:55: EPOCH 6 - PROGRESS: at 25.69% examples, 194492 words/s, in_qsize 11, out_qsize 5
INFO - 18:18:56: EPOCH 6 - PROGRESS: at 28.36% examples, 196365 words/s, in_qsize 12, out_qsize 0
INFO - 18:18:57: EPOCH 6 - PROGRESS: at 30.34% examples, 195272 words/s, in_qsize 12, out_qsize 5
INFO - 18:18:58: EPOCH 6 - PROGRESS: at 33.02% examples, 196278 words/s, in_qsize 13, out_qsize 0
INFO - 18:18:59: EPOCH 6 - PROGRESS: at 35.44% examples, 196270 words/s, in_qsize 13, out_qsize 0
INFO - 18:19:00: EPOCH 6 - PROGRESS: at 37.84% examples, 195965 words/s, in_qsize 12, out_qsize 3
INFO - 18:19:01: EPOCH 6 - PROGRESS: at 40.33% examples, 196507 words/s, in_qsize 13, out_qsize 0
INFO - 18:19:02: EPO

INFO - 18:20:10: worker thread finished; awaiting finish of 3 more threads
INFO - 18:20:10: worker thread finished; awaiting finish of 2 more threads
INFO - 18:20:10: worker thread finished; awaiting finish of 1 more threads
INFO - 18:20:10: worker thread finished; awaiting finish of 0 more threads
INFO - 18:20:10: EPOCH - 7 : training on 24789697 raw words (8436627 effective words) took 43.2s, 195161 effective words/s
INFO - 18:20:11: EPOCH 8 - PROGRESS: at 1.82% examples, 153016 words/s, in_qsize 13, out_qsize 1
INFO - 18:20:12: EPOCH 8 - PROGRESS: at 4.09% examples, 175969 words/s, in_qsize 11, out_qsize 2
INFO - 18:20:13: EPOCH 8 - PROGRESS: at 6.52% examples, 185593 words/s, in_qsize 12, out_qsize 1
INFO - 18:20:14: EPOCH 8 - PROGRESS: at 8.77% examples, 184977 words/s, in_qsize 10, out_qsize 6
INFO - 18:20:15: EPOCH 8 - PROGRESS: at 11.21% examples, 187413 words/s, in_qsize 14, out_qsize 2
INFO - 18:20:16: EPOCH 8 - PROGRESS: at 13.63% examples, 187960 words/s, in_qsize 13, out_q

...

INFO - 18:53:54: EPOCH 55 - PROGRESS: at 63.29% examples, 199607 words/s, in_qsize 14, out_qsize 0
INFO - 18:53:56: EPOCH 55 - PROGRESS: at 64.89% examples, 194156 words/s, in_qsize 12, out_qsize 4
INFO - 18:53:57: EPOCH 55 - PROGRESS: at 67.29% examples, 194249 words/s, in_qsize 12, out_qsize 5
INFO - 18:53:58: EPOCH 55 - PROGRESS: at 69.81% examples, 194785 words/s, in_qsize 13, out_qsize 0
INFO - 18:53:59: EPOCH 55 - PROGRESS: at 72.25% examples, 194815 words/s, in_qsize 13, out_qsize 0
INFO - 18:54:00: EPOCH 55 - PROGRESS: at 74.79% examples, 195391 words/s, in_qsize 10, out_qsize 0
INFO - 18:54:01: EPOCH 55 - PROGRESS: at 77.24% examples, 195135 words/s, in_qsize 13, out_qsize 0
INFO - 18:54:02: EPOCH 55 - PROGRESS: at 79.66% examples, 194971 words/s, in_qsize 13, out_qsize 7
INFO - 18:54:03: EPOCH 55 - PROGRESS: at 82.35% examples, 195615 words/s, in_qsize 14, out_qsize 0
INFO - 18:54:04: EPOCH 55 - PROGRESS: at 84.70% examples, 195608 words/s, in_qsize 11, out_qsize 3
INFO - 18:

INFO - 18:55:08: EPOCH 57 - PROGRESS: at 36.16% examples, 201475 words/s, in_qsize 14, out_qsize 0
INFO - 18:55:09: EPOCH 57 - PROGRESS: at 38.70% examples, 201290 words/s, in_qsize 14, out_qsize 1
INFO - 18:55:10: EPOCH 57 - PROGRESS: at 41.12% examples, 201392 words/s, in_qsize 14, out_qsize 0
INFO - 18:55:11: EPOCH 57 - PROGRESS: at 43.48% examples, 200379 words/s, in_qsize 14, out_qsize 6
INFO - 18:55:12: EPOCH 57 - PROGRESS: at 46.10% examples, 201422 words/s, in_qsize 13, out_qsize 0
INFO - 18:55:13: EPOCH 57 - PROGRESS: at 48.61% examples, 201495 words/s, in_qsize 13, out_qsize 0
INFO - 18:55:14: EPOCH 57 - PROGRESS: at 51.02% examples, 201666 words/s, in_qsize 13, out_qsize 1
INFO - 18:55:15: EPOCH 57 - PROGRESS: at 53.55% examples, 201636 words/s, in_qsize 13, out_qsize 0
INFO - 18:55:16: EPOCH 57 - PROGRESS: at 55.55% examples, 201160 words/s, in_qsize 14, out_qsize 6
INFO - 18:55:17: EPOCH 57 - PROGRESS: at 58.13% examples, 201576 words/s, in_qsize 11, out_qsize 5
INFO - 18:

INFO - 18:56:21: EPOCH 59 - PROGRESS: at 9.19% examples, 193554 words/s, in_qsize 13, out_qsize 2
INFO - 18:56:22: EPOCH 59 - PROGRESS: at 11.50% examples, 193206 words/s, in_qsize 14, out_qsize 4
INFO - 18:56:23: EPOCH 59 - PROGRESS: at 14.19% examples, 197250 words/s, in_qsize 13, out_qsize 0
INFO - 18:56:24: EPOCH 59 - PROGRESS: at 16.60% examples, 199049 words/s, in_qsize 14, out_qsize 0
INFO - 18:56:25: EPOCH 59 - PROGRESS: at 18.92% examples, 198860 words/s, in_qsize 12, out_qsize 3
INFO - 18:56:26: EPOCH 59 - PROGRESS: at 21.41% examples, 199563 words/s, in_qsize 9, out_qsize 5
INFO - 18:56:27: EPOCH 59 - PROGRESS: at 23.85% examples, 198662 words/s, in_qsize 13, out_qsize 6
INFO - 18:56:28: EPOCH 59 - PROGRESS: at 26.41% examples, 200454 words/s, in_qsize 13, out_qsize 0
INFO - 18:56:29: EPOCH 59 - PROGRESS: at 28.87% examples, 200748 words/s, in_qsize 13, out_qsize 0
INFO - 18:56:30: EPOCH 59 - PROGRESS: at 31.31% examples, 201104 words/s, in_qsize 13, out_qsize 0
INFO - 18:56

INFO - 18:57:40: EPOCH 60 - PROGRESS: at 97.37% examples, 199539 words/s, in_qsize 14, out_qsize 7
INFO - 18:57:41: worker thread finished; awaiting finish of 6 more threads
INFO - 18:57:41: worker thread finished; awaiting finish of 5 more threads
INFO - 18:57:41: worker thread finished; awaiting finish of 4 more threads
INFO - 18:57:41: worker thread finished; awaiting finish of 3 more threads
INFO - 18:57:41: worker thread finished; awaiting finish of 2 more threads
INFO - 18:57:41: worker thread finished; awaiting finish of 1 more threads
INFO - 18:57:41: worker thread finished; awaiting finish of 0 more threads
INFO - 18:57:41: EPOCH - 60 : training on 24789697 raw words (8434342 effective words) took 41.9s, 201282 effective words/s
INFO - 18:57:41: Word2Vec lifecycle event {'msg': 'training on 1487381820 raw words (506055495 effective words) took 2555.7s, 198010 effective words/s', 'datetime': '2022-01-26T18:57:41.253439', 'gensim': '4.1.2', 'python': '3.8.6 (tags/v3.8.6:db45529,

Time to train the model: 42.6 mins


In [17]:
w2v_model.save("word2vec-Yelp-real-500Kcomment-v2-100vec.model")

INFO - 17:19:29: Word2Vec lifecycle event {'fname_or_handle': 'word2vec-Yelp-real-500Kcomment-v2-100vec.model', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2022-01-27T17:19:29.204451', 'gensim': '4.1.2', 'python': '3.8.6 (tags/v3.8.6:db45529, Sep 23 2020, 15:52:53) [MSC v.1927 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'saving'}
INFO - 17:19:29: storing np array 'vectors' to word2vec-Yelp-real-500Kcomment-v2-100vec.model.wv.vectors.npy
INFO - 17:19:29: storing np array 'syn1neg' to word2vec-Yelp-real-500Kcomment-v2-100vec.model.syn1neg.npy
INFO - 17:19:30: not storing attribute cum_table
INFO - 17:19:31: saved word2vec-Yelp-real-500Kcomment-v2-100vec.model


--------

TEST

In [ ]:
pathTip = 'D:\\t2n_projects\\yelp\\yelp_academic_dataset_tip.json'
pathCheckin = 'D:\\t2n_projects\\yelp\\yelp_academic_dataset_checkin.json'
pathReview = 'D:\\t2n_projects\\yelp\\original-data\\yelp_academic_dataset_review.json'
pathUser = 'D:\\t2n_projects\\yelp\\yelp_academic_dataset_user.json'
pathBusiness = 'D:\\t2n_projects\\yelp\\yelp_academic_dataset_business.json'

In [ ]:
from sklearn.cluster import KMeans

import numpy as np

np.shape(w2v_model.wv.vectors)

w2v_model.wv['good']

# word_vectors = Word2Vec.load("../preprocessing_and_embeddings/word2vec.model").wv
word_vectors = w2v_model.wv
model = KMeans(n_clusters=5, max_iter=1000, random_state=True, n_init=50).fit(X=word_vectors.vectors)
positive_cluster_center = model.cluster_centers_[0]
negative_cluster_center = model.cluster_centers_[1]

with open('kmean-5cluster.pkl','wb') as f:
    pickle.dump(model,f)

word_vectors.vectors

w2v_model = Word2Vec.load('word2vec.model')

w2v_model.wv.most_similar(positive=['Trump'], topn=10)

dfContent = pd.read_csv('reviewContent', sep='\t', header=None)

dfContent.groupby(1).size()

dfContent.rename(columns={0:'userId', 2:'date', 3:'text'})

In [ ]:
splitBigJson(pathReview)

pathReview = 'D:\\t2n_projects\\yelp\\reviews\\yelp_academic_dataset_review'

lstDF = []
for i in range(10):
    pathN = pathReview + str(i) + '.json'
    print('read file', pathN)
    with open(pathN, encoding="utf8") as file:
        data = [json.loads(line) for line in file]
    dfTmp = pd.DataFrame(data[0])
    lstDF.append(dfTmp)
dfReview = pd.concat(lstDF, ignore_index=True)
dfReview = dfReview.loc[:, ~dfReview.columns.str.contains('^Unnamed')]

len(dfReview)

dfReview.to_pickle('sampleReview5M.pkl', compression='gzip')